In [2]:
## Model.py
import torch
import torch.nn as nn
import math

class LayerNormalization(nn.Module):

  def __init__(self, eps:float=10**-6)->None:
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1)) #alpha is learnable paramter
    self.bias = nn.Paramter(torch.zeros(1)) #bias is a learnable paramter

  def forward(self, x):
    # x: (batch, seq_len,from hidden_size)
    # Keep dimension for broadcasting
    mean = x.mean(dim = -1, keepdim= True) # (batch, seq_len ,1)
    std = x.std(dim=-1, keepdim=True) # (batch, seq_len ,1)
    # eps is to prevent dividing by zero or when std is very small
    return self.alpha * (x - mean )/( std + self.eps ) + self.bias


class FeedForwardBlock(nn.Module):

  def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff) #w1 and b1
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class InputEmbedding(nn.Module):

  def __init__(self, d_model: int, vocab_size: int) -> None:
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    # (batch, seq_len) --> (batch, seq_len, d_model)
    # Multiply by sqrt(d_model) to scale the embeddings according to the paper
    return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):

  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    # Create a matrix of shape (seq_len, d_model) one pe for each word
    pe = torch.zeros(seq_len, d_model)
    # Create a vector of shape (seq_len)
    position = torch.arange(0, seq_len, dtype =torch.float).unsqueeze(1) #(seq_len, 1)
    # Create a vector of shape (d_model)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/ d_model))
    # Apply sine to even indices
    pe[:, 0::2] = torch.sin(position* div_term) # sin(position *(10000 ** (2i/ d_model)))
    # Apply cosine to add indices
    pe[:, 1::2] = torch.cos(position * div_term) #cos(position * (10000 ** (2i/ d_model)))
    # Add a batch dimension to the positional encoding
    pe = pe.unsqueeze(0) # (1, seq_len, d_model)
    # Register the positional encoding as a buffer
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad(False) # (batch, seq_len, d_model)
    return self.dropout(x)

class ResidualConnection(nn.Module):

  def __init(self, dropout: float) -> None:
    self.droupout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

  def __init__(self, d_model: int, h: int, dropout: float) -> None:
    super().__init__()
    self.d_model = d_model # Embedding vector size
    self.h = h # NUmber of heads
    # Make sure d_model is divisible by h
    assert d_model % h == 0, "d_model is not divisible by h"

    self.d_k = d_model//h # Dimension of vector seen by each hand
    self.w_q = nn.Linear(d_model, d_model, bias=False) #Wq
    self.w_k = nn.Linear(d_model, d_model, bias=False) #Wk
    self.w_v = nn.Linear(d_model, d_model, bias=False) #Wv
    self.w_o = nn.Linear(d_model, d_model, bias=False) #Wo
    self.dropout =  nn.Dropout(dropout)


  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]
    # Just apply the formula from the paper
    # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2, -1))/math.sqrt(d_k)
    if mask is not None:
      # Wite a very low value (value indicating -inf) to the positions where mask == 0
      attention_scores.masked_fill_(mask==0, -1e9)
    attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len)  # Apply Softmax
    if dropout is not None:
      attention_scores = dropout(attention_scores)
    # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
    # return attention scores which can be used for visualization
    return (attention_scores @ value), attention_scores

  def forward(self, q, k ,v, mask):
    query = self.w_q(q) #(batch, seq_len, d_model) --> (batch, seq_len, d_model)
    key = self.w_k(k) #(batch, seq_len, d_model) --> (batch, seq_len, d_model)
    value = self.w_v(v) #(batch, seq_len, d_model) --> (batch, seq_len, d_model)

    # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

    # Calculate attention
    x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask. self.dropout)

    # Combine all the heads together
    # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
    x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

    # Multiply by Wo
    # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
    return self.w_o(x)


class EncoderBlock(nn.Module):

  def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock):
    super().__int__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections  = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, lambda x: self.feed_forward_block)
    return x

class Encoder(nn.Module):

  def __init__(self, layers: nn.ModuleList)-> None:
    super().__int__()
    self.layers = layers
    self.norm = LayerNormalization()


  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)


class DecoderBlock(nn.Module):

  def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, residual_connections:nn.ModuleList):
    super().__int__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def foward(self, x, encoder_output, src_mask, tgt_mask):
      x = residual_connections[0](x, lambda: self.self_attention_block(x, x, x, tgt_mask))
      x = residual_connections[1](x, lambda: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
      x = residual_connections[2](x, feed_forward_block)
      return x

class Decoder(nn.Module):

  def __init__(self, layers:nn.ModuleList)->None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
      return self.norm(x)

class ProjectionLayer(nn.Module):

  def __init__(self, d_model, vocab_size) -> None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x)-> None:
    # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
    return torch.log_softmax(self.proj(x), dim=-1)

class Transformer(nn.Module):

  def __int__(self, encoder:Encoder, decoder: Decoder, src_embed: InputEmbedding, tgt_embed:InputEmbedding, src_pos:PositionalEncoding, tgt_pos:PositionalEncoding, projection_layer:ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    # (batch, seq_len, d_model)
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
    # (batch, seq_len, d_model)
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    # (batch, seq_len, vocab_size)
    return self.projection_layer(x)

  def build_transformer(src_vocab_size: int, tgt_vocab_size:int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout : float = 0.1, d_ff:int=2048)-> None:

    #Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
      encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
      feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
      encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
      encoder_block.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
      decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
      decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
      feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
      decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
      decoder_blocks.append(decoder_blocks)


    # Create the encoder and decoder
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialise the parameters
    for p in transformer.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

    return transformer

In [3]:
from pathlib import Path

def get_config():
  return {
      "batch_size": 2048,
      "num_epochs": 20,
      "lr": 10**-4,
      "seq_len":350,
      "d_model":512,
      "lang_src": "en",
      "lang_tgt":"it",
      "model_folder":"weights",
      "model_basename": "tmodel_",
      "preload": True,
      "tokenizer_file": "tokenizer_{0}.json",
      "experiment_name":"runs/tmodel"
  }

def get_weights_file_path(config, epoch:str):
  model_folder = config["model_folder"]
  model_basename = config["model_basename"]
  model_filename = f"{model_basename}{epoch}.pt"

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class BilingualDataset(Dataset):

  def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
    super().__int__()
    self.seq_len = seq_len
    self.sd = ds
    self.tokenizer_src = tokenizer_src
    self.tokenizer_tgt = tokenizer_tgt
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang

    self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype = torch.int64)
    self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype = torch.int64)
    self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype = torch.int64)

  def __len__(self):
    return len(self.ds)

  def __getitem__(self, idx):
    src_target_pair = self.ds[idx]
    src_text = src_target_pair['translation'][self.src_lang]
    tgt_text = src_target_pair['translation'][self.tgt_lang]

    #Transform  the text into tokens
    enc_input_tokens = self.tokenizer_src.encode(src_text).ids
    dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

    # Add sos, eos and pasding to each sentence
    enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) -2
    dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) -1

    # Make sure the number of padding tokens is not negative
    if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
      raise ValueError("Sentence is too long")

    # Add <s> and </s> token
    encoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(enc_input_tokens, dtype = torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token]*enc_num_padding_tokens,dtype=torch.int64),
        ],
        dim=0,
    )

    # Add only <s> token
    decoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype = torch.int64),
            torch.tensor([self.pad_token]*dec_num_padding_tokens,dtype=torch.int64),
        ],
        dim=0,
    )

    label = torch.cat(
        [
              torch.tensor(dec_input_tokens, dtype = torch.int64),
              self.eos_token,
              torch.tensor([self.pad_token]*dec_num_padding_tokens,dtype=torch.int64),

        ],
        dim=0,
    )

    # Double check the size of the tensors to make sure they are all seq_len long
    assert encoder_input.size(0)==self.seq_len
    assert decoder_input.size(0)==self.seq_len
    assert label.size(0)==self.seq_len

    return {
        "encoder_input": encoder_input, #(seq_len)
        "decoder_input": decoder_input, #(seq_len)
        "encoder_mask": (encoder_input !=self.pad_tokens).unsqueeze(0).unsqueeze(0).int(), #(1, 1, seq_len)
        "decoder_mask": (decoder_input !=self.pad_tokens).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), #(1, 1, seq_len)
        "label": label, #(seq_len)
        "src_text":src_text,
        "tgt_text":tgt_text,
    }


def causal_mask(size):
  mask = torch.triu(torch.ones((1,size,size)), diagonal=1).type(torch.int)
  return mask==0

In [6]:
# from model import build_transformer
# from dataset import BilingualDataset, causal_mask
# from config import get_config, get_weights_file_path

import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import DataSet, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm
import os
from pathlib import Path

## Hugging face dataset and tokenizers
from datasets import load_datasets
from tokenizers import Tokenizer
from tokenizer.models import WordLevel
from tokenizer.trainers import WordLevelTrainer
from tokenizer.pre_tokenizers import Whitespace
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
  sos_idx = tokenizer_tgt.token_to_id('[SOS]')
  eos_idx = tokenizer_tgt.token_to_id('[EOS]')

  # Precompute the encoder output and reuse it for every step
  encoder_output = model.encode(source, source_mask)
  #Initialise the decoder input with the sos token
  decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
  while True:
    if decoder_input.size(1) == max_len:
      break

    decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

    # Calculate output
    out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

    # get the next token
    prob = model.project(out[:,-1])
    _, next_word = torch.max(prob, dim =1)
    decoder_input = torch.cat(
        [decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item()).to(device)], dim=1
    )

    if next_word == eos_idx:
      break
  return decoder_input_squeeze(0)

def run_validation(model, validation_ds, tokenizer_src,tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
  model.eval()
  count = 0

  source_texts = []
  expected = []
  predicted = []

  try:
    # get the console window width
    with os.popen('stty size', 'r') as console:
      _, console_with = console.read().split()
      console_with = int(console_with)

  except:
    # If we cnat get console width then use a default of 80
    console_width = 80

  with torch.no_grad():
    for batch in validation_ds:
      count +=1
      encoder_input = batch['encoder_input'].to(device) #(b, seq_len)
      encoder_mask = batch['encoder_mask'].to(device) #(b, 1, 1, seq_len)

      assert encoder_input.size(0)===1, "Batch size must be 1 for validation"

      model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

      source_text = batch["src_text"][0]
      target_text = batch["tgt_text"][0]
      model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

      source_texts.append(source_text)
      expected.append(target_text)
      predicted.append(model_out_text)

      # Print the source, target and the model output
      print_msg('-'*console_width)
      print_msg(f"{f'SOURCE: ':>12}{source_text}")
      print_msg(f"{f'TARGET: ':>12}{target_text}")
      print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

      if count == num_examples:
        print_msg('-'*console_width)
        break


  if writer:
    # Evaluate the character error rate
    # Compute the char error rate
    metric = torchmetrics.CharErrorRate()
    cer = metric(predicted, expected)
    writer.add_scalar('validation cer', cer, global_step)
    writer.flush()

    # Evaluate the Word error rate
    # Compute the Word error rate
    metric = torchmetrics.WordErrorRate()
    wer = metric(predicted, expected)
    writer.add_scalar('validation wer', wer, global_step)
    writer.flush()

    # Compute the BLEU rate
    metric = torchmetrics.BLEUScore()
    bleu = metric(predicted, expected)
    writer.add_scalar('validation bleu', bleu, global_step)
    writer.flush()


def get_all_sentences(ds, lang):
  for item in ds:
    yield item['translation'][lang]

def get_or_build_tokenizer(config, ds, lang):
  tokenizer_path = Path(config['tokenizer_file'].format(lang))
  if not Path.exists(tokenzier_path):
    # Code taken mostly from: https://huggingface.co/docs/tokenziers/quicktour
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = WhiteSpace()
    trainer = WordLevelTrainer(special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
    tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
    tokenizer.save(str(tokenzier_path))
  else:
    tokenizer = Tokenizer.from_file(str(tokenizer_path))
  return tokenizer

def get_ds(config):
  # Only train split so we divide it oursleves.
  ds_raw = load_dataset('opus_books', f"{config['lang_src']}-{config['lang_tgt']}", split='train')

  # Build tokenizers
  tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
  tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

  # Keep 90% for training, 10 % for validation
  train_ds_size =  int(0.9 * len (ds_raw))
  val_ds_size =  len(ds_raw) - train_ds_size
  train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

  train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'],config['lang_tgt'],)
  val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'],config['lang_tgt'],)

  max_len_src = 0
  max_len_tgt = 0

  for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

  print(f'Max length of source sentence: {max_len_src}')
  print(f'Max length of target sentence: {max_len_tgt}')

  train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
  val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

  return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt


def get_model(config, vocab_src_len, vocab_tgt_len):
  model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'],config['seq_len'],d_model=config['d_model'])
  return model

def train_model(config):
  # Define the device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("Using device:" ,device)

  # Make sure the weights folder exists
  Path(config['model_folder']).mkdir(parents=True, exists_ok=True)

  train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
  model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

  # Tensorboard
  writer = SummaryWriter(config['experiment_name'])

  optimizer = torch.optim.Adam(model.parameters(), lr = config['lr'], eps=1e-9)

  # If the user specified a model to preload before training, load it
  initial_epoch = 0
  global_step = 0
  if config['preload']:
    model_filename = get_weights_file_path(config, config['preload'])
    print(f'Preloading model {model_filename}')
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
    inital_epoch = state['epoch'] + 1
    optimizer.load_state_dict(state['optimizer_state_dict'])
    global_step = state['global_step']
    print('Preload')


  loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)

  for epoch in range(initial_epoch, config['num_epochs']):
    torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
    for batch in batch_iterator:

      encoder_input = batch['encoder_input'].to(device) #(b, seq_len)
      decoder_input = batch['decoder_input'].to(device) #(B, seq_len)
      encoder_mask = batch['encoder_mask'].to(device) #(B, 1, 1, seq_len)
      decoder_mask = batch['decoder_mask'].to(device) #(B, 1, seq_len, seq_len)

      # Run the tenosrs through the encoder, decoder and the projection layer
      encoder_output = model.encode(encoder_input, encoder_mask) #(B, seq_len, d_model)
      decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) #(B, seq_len, d_model)
      proj_output = model.project(decoder_output) # (B, seq_len, vocab_size)

      # Compare the output with the label
      label = batch['label'].to(device) # (B, seq_len)

      # Compute the output with the label
      loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
      batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

      # Log the loss
      writer.add_scalar('train loss', loss.item(), global_step)
      writer.flush()

      #Backpropogate the loss
      loss.backward()

      #Update the weights
      optimizer.step()
      optimizer.zero_grad(set_to_none=True)

      global_step +=1

    # Run validation at th end of each epoch
    run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, print_msg, global_step, writer, num_examples=2)

    #Save themodel at thened of every epoch
    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save({
        'epoch': epoch,
        'model_state_dict':model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'global_step':global_step
    }, model_filename)

if __name__=='__main__':
  warnings.filterwarnings("ignore")
  config = get_config()
  train_model(config)

SyntaxError: ignored

In [ ]:
# from config import get_config
cfg = get_config()
cfg['batch_size'] = 6
cfg['preload'] = None
cfg['num_epochs'] = 10

# from train import train_model
''
train_model(cfg)